In [2]:
import pandas as pd
import numpy as np
import pathlib
from tqdm.notebook import tqdm
import librosa
import awkward as ak
import matplotlib.pyplot as plt
from IPython.display import Audio 
from collections import defaultdict
from scipy.signal import decimate


from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [3]:
# Read the TS continous values
X_train = np.load("../../RavdessAudioOnlyNumpy__X_train.npy")
X_test = np.load("../../RavdessAudioOnlyNumpy__X_test.npy")
print(X_train.shape, X_test.shape)

(1828, 304304) (634, 304304)


Clean the two numpy arrays deleting the columns found in the test set with null values

In [4]:
# Individua le colonne con valori nulli
col_with_nulls = np.isnan(X_train).any(axis=0)

# Ottieni gli indici delle colonne con valori nulli
indices_with_nulls = np.where(col_with_nulls)[0]

X_train = np.delete(X_train, indices_with_nulls, axis=1)
X_test = np.delete(X_test, indices_with_nulls, axis=1)

In [5]:
X_train.shape

(1828, 140941)

In [6]:
X_test.shape

(634, 140941)

Read the dataframe for train and test containing categorical data, then define 2 array containing the emotion value for both train and test

In [7]:
df_Y_train=pd.read_csv('../../RavdessAudioOnlyNumpy__Y_train.csv')
df_Y_test=pd.read_csv('../../RavdessAudioOnlyNumpy__Y_test.csv')

In [8]:
#LabelEncode categorical features
from sklearn import preprocessing

le=preprocessing.LabelEncoder()
c = df_Y_train.select_dtypes(include=['object']).columns.tolist()
for i in c:
    df_Y_train[i]=le.fit_transform(df_Y_train[i])

for i in c:
    df_Y_test[i]=le.fit_transform(df_Y_test[i])

In [9]:
y_train=np.array(df_Y_train['emotion'])
y_test=np.array(df_Y_test['emotion'])

Data Normalization

In [10]:
from tslearn.preprocessing import TimeSeriesScalerMinMax
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train).reshape(X_train.shape[0], X_train.shape[1])
X_test = scaler.fit_transform(X_test).reshape(X_test.shape[0], X_test.shape[1])

In [11]:
X_train.shape

(1828, 140941)

Approximation of time series

In [12]:
from tslearn.piecewise import PiecewiseAggregateApproximation
from tslearn.piecewise import SymbolicAggregateApproximation
from tslearn.piecewise import OneD_SymbolicAggregateApproximation
from pyts.approximation import DiscreteFourierTransform

n_coefs = 1000

dft = DiscreteFourierTransform(n_coefs=n_coefs)
ts_dft_X_train = dft.fit_transform(X_train) 
ts_dft_X_test = dft.fit_transform(X_test) 

Shapelet Classifiers

In [13]:
from keras.optimizers import Adagrad
from tslearn.shapelets import ShapeletModel
from tslearn.shapelets import grabocka_params_to_shapelet_size_dict

In [14]:
ts_dft_X_train.shape

(1828, 1000)

In [15]:
n_ts, ts_sz = ts_dft_X_train.shape
n_classes = len(set(y_train))

# Set the number of shapelets per size 
shapelet_sizes = grabocka_params_to_shapelet_size_dict(n_ts=n_ts, # this function that take n. of TS, n. T stamps, number of classe, learnig rate an it returns a set of suggested sizes to look for the shapelets
                                                       ts_sz=ts_sz,
                                                       n_classes=n_classes,
                                                       l=0.1,
                                                       r=1)

print('n_ts', n_ts)
print('ts_sz', ts_sz)
print('n_classes', n_classes)
print('shapelet_sizes', shapelet_sizes)

n_ts 1828
ts_sz 1000
n_classes 8
shapelet_sizes {100: 7}


In [16]:
shp_clf = ShapeletModel(n_shapelets_per_size=shapelet_sizes,
                        optimizer="sgd",
                        weight_regularizer=.01,
                        max_iter=200,
                        verbose=1)
shp_clf.fit(ts_dft_X_train, y_train)

d:\Anaconda\lib\site-packages\tslearn\shapelets\shapelets.py:354: FutureWarning: The default value for 'scale' is set to False in version 0.4 to ensure backward compatibility, but is likely to change in a future version.
  warnings.warn("The default value for 'scale' is set to False "


Epoch 1/200
8/8 [==============================] - 1s 35ms/step - loss: 5460246016.0000 - categorical_accuracy: 0.1373 - categorical_crossentropy: 5453830144.0000
Epoch 2/200
8/8 [==============================] - 0s 35ms/step - loss: 2493587456.0000 - categorical_accuracy: 0.1373 - categorical_crossentropy: 2488486656.0000
Epoch 3/200
8/8 [==============================] - 0s 38ms/step - loss: 3372257536.0000 - categorical_accuracy: 0.1417 - categorical_crossentropy: 3369141248.0000
Epoch 4/200
8/8 [==============================] - 0s 32ms/step - loss: 1613273984.0000 - categorical_accuracy: 0.1526 - categorical_crossentropy: 1611730560.0000
Epoch 5/200
8/8 [==============================] - 0s 37ms/step - loss: 1283577472.0000 - categorical_accuracy: 0.1187 - categorical_crossentropy: 1282666496.0000
Epoch 6/200
8/8 [==============================] - 0s 33ms/step - loss: 1980539136.0000 - categorical_accuracy: 0.1187 - categorical_crossentropy: 1979659264.0000
Epoch 7/200
8/8 [=====

LearningShapelets(max_iter=200, n_shapelets_per_size={100: 7}, verbose=1,
                  weight_regularizer=0.01)

In [17]:
y_pred = shp_clf.predict(ts_dft_X_test)
print(classification_report(y_test, y_pred))

3/3 [==============================] - 0s 22ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        96
           1       0.00      0.00      0.00        97
           2       0.07      0.16      0.10        50
           3       0.00      0.00      0.00        98
           4       0.16      0.87      0.28        98
           5       0.00      0.00      0.00        49
           6       0.00      0.00      0.00        98
           7       0.00      0.00      0.00        48

    accuracy                           0.15       634
   macro avg       0.03      0.13      0.05       634
weighted avg       0.03      0.15      0.05       634



d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Time Series Classifier

In [18]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(ts_dft_X_train, y_train)

y_pred = clf.predict(ts_dft_X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.18      0.38      0.25        96
           1       0.13      0.13      0.13        97
           2       0.12      0.06      0.08        50
           3       0.13      0.14      0.14        98
           4       0.19      0.18      0.19        98
           5       0.14      0.06      0.08        49
           6       0.23      0.18      0.20        98
           7       0.18      0.04      0.07        48

    accuracy                           0.17       634
   macro avg       0.16      0.15      0.14       634
weighted avg       0.17      0.17      0.16       634



In [ ]:
clf = KNeighborsClassifier(metric='dtw_sakoechiba') # we can use this type of KNN for TS sèecifing the global constraints
clf.fit(ts_dft_X_train, y_train)
y_pred = clf.predict(ts_dft_X_test)
print(classification_report(y_test, y_pred))

In [21]:
clf = KNeighborsClassifier(metric='euclidean') # we can use this type of KNN for TS sèecifing the global constraints
clf.fit(ts_dft_X_train, y_train)
y_pred = clf.predict(ts_dft_X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.18      0.38      0.25        96
           1       0.13      0.13      0.13        97
           2       0.12      0.06      0.08        50
           3       0.13      0.14      0.14        98
           4       0.19      0.18      0.19        98
           5       0.14      0.06      0.08        49
           6       0.23      0.18      0.20        98
           7       0.18      0.04      0.07        48

    accuracy                           0.17       634
   macro avg       0.16      0.15      0.14       634
weighted avg       0.17      0.17      0.16       634



In [22]:
clf = KNeighborsClassifier(metric='manhattan') # we can use this type of KNN for TS sèecifing the global constraints
clf.fit(ts_dft_X_train, y_train)
y_pred = clf.predict(ts_dft_X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.19      0.33      0.24        96
           1       0.19      0.20      0.19        97
           2       0.08      0.04      0.05        50
           3       0.13      0.16      0.15        98
           4       0.16      0.18      0.17        98
           5       0.07      0.02      0.03        49
           6       0.17      0.15      0.16        98
           7       0.00      0.00      0.00        48

    accuracy                           0.16       634
   macro avg       0.12      0.14      0.12       634
weighted avg       0.14      0.16      0.15       634



In [19]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=8, random_state=42)
clf.fit(ts_dft_X_train, y_train)

y_pred = clf.predict(ts_dft_X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.06      0.02      0.03        96
           1       0.35      0.06      0.11        97
           2       0.05      0.04      0.04        50
           3       0.08      0.01      0.02        98
           4       0.17      0.49      0.25        98
           5       0.06      0.02      0.03        49
           6       0.17      0.32      0.22        98
           7       0.12      0.12      0.12        48

    accuracy                           0.15       634
   macro avg       0.13      0.14      0.10       634
weighted avg       0.15      0.15      0.11       634



In [20]:
clf = KNeighborsTimeSeriesClassifier()
clf.fit(ts_dft_X_train, y_train)
y_pred = clf.predict(ts_dft_X_test)
print(classification_report(y_test, y_pred))

KeyboardInterrupt: 